# Data Preparation

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
train=pd.read_csv('y_train.csv')
custinfo=pd.read_csv('demo.csv', skipinitialspace=True)
cardinfo=pd.read_csv('card_info.csv')
transaction=pd.read_csv('cc_txn.csv')
balance=pd.read_csv('sa_bal.csv')
inandout=pd.read_csv('dtxn.csv')

In [ ]:
train.head()

In [ ]:
train['label'].value_counts()

In [ ]:
ax = sns.countplot(x="label", data=train, palette="hls")

In [ ]:
custinfo.info()

In [ ]:
custinfo.head()

In [ ]:
cardinfo.info()

In [ ]:
cardinfo.head()

In [ ]:
transaction.info()

In [ ]:
transaction.head()

In [ ]:
transaction['txn_dt']=pd.to_datetime(transaction['txn_dt'])
transaction.info()


In [ ]:
transaction[['txn_amt', 'card_no']]=transaction[['txn_amt', 'card_no']].astype(np.int64)


In [ ]:
transaction.info()

In [ ]:
transaction['year'] = pd.DatetimeIndex(transaction['txn_dt']).year
transaction['month'] = pd.DatetimeIndex(transaction['txn_dt']).month
transaction['date'] = pd.DatetimeIndex(transaction['txn_dt']).day
transaction['quarter'] = pd.DatetimeIndex(transaction['txn_dt']).quarter
transaction.head()

In [ ]:
transaction1=transaction.pivot_table(index=['user_id', 'card_no'], columns=['mcc'], aggfunc='size', fill_value=0).reset_index().rename_axis(None, axis=1)
transaction1.head()


In [ ]:
transaction2=transaction.pivot_table(index=['user_id', 'card_no'], columns=['quarter'], aggfunc='size', fill_value=0).reset_index().rename_axis(None, axis=1)
transaction2.columns = ['user_id', 'card_no', 'Q1', 'Q2', 'Q3', 'Q4']
transaction2.head()

In [ ]:
transaction3=(transaction.groupby(['user_id', 'card_no'], as_index=False).agg({'txn_amt': ['mean', 'min', 'max', 'count']}))
transaction3.columns = ['user_id', 'card_no', 'txn_amt_avg', 'txn_amt_min', 'txn_amt_max', 'txn_count']
transaction3.head()



In [ ]:
transaction4=pd.merge(transaction3, transaction2, left_on=['user_id', 'card_no'], right_on=['user_id', 'card_no'])
transaction4.head()

In [ ]:
transactionfinal=pd.merge(transaction4, transaction1, left_on=['user_id', 'card_no'], right_on=['user_id', 'card_no'])
transactionfinal.info()
transactionfinal['txn_amt_avg']=transactionfinal['txn_amt_avg'].astype(np.int64)

In [ ]:
transactionfinal.head()

In [ ]:
balancefinal=balance.pivot_table(index=['user_id'], values=['max_sa_bal'], aggfunc='mean').reset_index()
balancefinal.columns = ['user_id','avg_max_sa_bal']
balancefinal.head()

In [ ]:
inandout['amt_net'] = inandout['amt_in'] - inandout['amt_out']


In [ ]:
inandout.head()

In [ ]:
inandoutfinal= inandout.pivot_table(index=['user_id'], values=['amt_net'], aggfunc='mean').reset_index()
inandoutfinal.columns = ['user_id','avg_net_in_out']
inandoutfinal.head()

In [ ]:
bankacc=pd.merge(balancefinal, inandoutfinal, left_on=['user_id'], right_on=['user_id'])
bankacc.info()

In [ ]:
bankacc.head()

In [ ]:
custinfo['account_start_date']=pd.to_datetime(custinfo['account_start_date'])
custinfo['year'] = pd.DatetimeIndex(custinfo['account_start_date']).year
custinfo['tenure']= 2017 - custinfo['year'] +1
custinfo['age']= 2017 - custinfo['birth_year'] 
custinfo.head()

In [ ]:
df=pd.merge(train, custinfo, left_on="user_id", right_on="user_id", how="left")
df=pd.merge(df, cardinfo, left_on="user_id", right_on="user_id", how="left")
df.head()


In [ ]:
df.head()

In [ ]:
df=pd.merge(df, transactionfinal, left_on=['user_id', 'card_no'], right_on=['user_id', 'card_no'], how="left")

In [ ]:
df=pd.merge(df, bankacc, left_on="user_id", right_on="user_id")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.sort_values(['txn_count', 'txn_amt_avg'], ascending=[False, False])

In [ ]:
df1=df.drop_duplicates('user_id')
df1.info()

In [ ]:
df1.head()

In [ ]:
pd.options.mode.chained_assignment = None 
gender = {'M': 1,'F': 2}
df1['gender'] = [gender[item] for item in df1['gender']] 
df1['gender']=df1['gender'] .astype('Int64')
df1['label']=df1['label'] .astype('Int64')
df1['birth_year']=df1['birth_year'] .astype('Int64')
df1['marital_status']=df1['marital_status'] .astype('Int64')
df1['family_income_segment_code']=df1['family_income_segment_code'] .astype('Int64')
df1['individual_income_segment_code']=df1['individual_income_segment_code'] .astype('Int64')
df1['bill_cyc']=df1['bill_cyc'] .astype('Int64')
df1['age']=df1['age'] .astype('Int64')
df1.info()


In [ ]:
df1.head()

In [ ]:
df1.isnull().sum()

In [ ]:
import numpy as np
marital_status = df1.marital_status.values
marital_status = np.reshape(marital_status,(-1,1))
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values = np.nan , strategy='most_frequent')
imp.fit(marital_status)
df1['marital_status'] = imp.transform(marital_status)
df1[df1['marital_status'].isnull()]

In [ ]:
individual_income_segment_code = df1.individual_income_segment_code.values
individual_income_segment_code = np.reshape(individual_income_segment_code,(-1,1))
imp = SimpleImputer(missing_values = np.nan , strategy='most_frequent')
imp.fit(individual_income_segment_code)
df1['individual_income_segment_code'] = imp.transform(individual_income_segment_code)
df1[df1['individual_income_segment_code'].isnull()]

In [ ]:
family_income_segment_code = df1.family_income_segment_code.values
family_income_segment_code = np.reshape(family_income_segment_code,(-1,1))
imp = SimpleImputer(missing_values = np.nan , strategy='most_frequent')
imp.fit(family_income_segment_code)
df1['family_income_segment_code'] = imp.transform(family_income_segment_code)
df1[df1['family_income_segment_code'].isnull()]

In [ ]:
corr_matrix = df1.corr()
print(corr_matrix['label'].sort_values(ascending=False))

In [ ]:
df1['label'].value_counts()
ax = sns.countplot(x="label", data=df1, palette="hls")

In [ ]:
count_ac = len(df1[df1['label']==0])
count_no_ac = len(df1[df1['label']==1])
pct_of_no_ac = count_no_ac/(count_no_ac+count_ac)
print("percentage of InActive user is", pct_of_no_ac*100)
pct_of_ac = count_ac/(count_no_ac+count_ac)
print("percentage of Active user", pct_of_ac*100)

In [ ]:
df1.groupby('label').mean()

In [ ]:
%matplotlib inline
pd.crosstab(df1.individual_income_segment_code,df1.label).plot(kind='bar')
plt.title('Churn Frequency for Individual Job Segment')
plt.xlabel('individual_income_segment_code')
plt.ylabel('Frequency of Churn')
plt.savefig('individual_income_segment_code')

In [ ]:
%matplotlib inline
pd.crosstab(df1.family_income_segment_code,df1.label).plot(kind='bar')
plt.title('Churn Frequency for Family Job Segment')
plt.xlabel('individual_income_segment_code')
plt.ylabel('Frequency of Churn')
plt.savefig('family_income_segment_code')

In [ ]:
%matplotlib inline
pd.crosstab(df1.gender,df1.label).plot(kind='bar')
plt.title('Churn Frequency for Gender')
plt.xlabel('individual_income_segment_code')
plt.ylabel('Frequency of Churn')
plt.savefig('gender')

In [ ]:
df1 = df1.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64'])
!pip install imblearn

In [ ]:
df1.columns.values

# Under Development

In [ ]:
X = df1.loc[:, df1.columns != 'label']
y = df1.loc[:, df1.columns == 'label'].astype('int')
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['label'])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['label']==0]))
print("Number of subscription",len(os_data_y[os_data_y['label']==1]))
print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['label']==0])/len(os_data_X))
print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['label']==1])/len(os_data_X))

In [ ]:
df1.info()

In [ ]:

final_vars=df1.columns.values.tolist()
y=['label']
X=[i for i in final_vars if i not in y]
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
cols=['tenure','age','txn_amt_avg','txn_amt_min','txn_amt_max','txn_count','Q1','Q2','Q3','Q4','mcc_cat10','mcc_cat13','mcc_cat16','mcc_cat5','mcc_cat8','mcc_cat9',] 
X=os_data_X[cols]
y=os_data_y['label']

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit(method='lbfgs',)
result.summary2()

In [ ]:
from sklearn.model_selection import train_test_split 
train, test = train_test_split(df2, test_size = 0.3)
 
train_y = train['label'].astype(int)
test_y = test['label']
 
train_x = train
train_x.pop('label')
test_x = test
test_x.pop('label')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))